# Plotting Inflow into Reservoirs

This script will plot inflow into the major reservoirs of the State Water Project. For most, this is the sum of inputs from all tributaries. For the Thermalito Afterbay, the inflow was calculated as the difference between Oroville's outflow and the river discharge immediately south of Thermalito's offshoot.

The information is plotted using matplotlib's `pyplot` library and premade widgets from their `widgets` library. Information on the specifics of these widgets is available in the [examples](https://matplotlib.org/gallery/widgets/slider_demo.html#sphx-glr-gallery-widgets-slider-demo-py) and [API](https://matplotlib.org/api/widgets_api.html) documentation.

In [9]:
%matplotlib widget
import csv
import datetime
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from matplotlib.widgets import Slider, Button, RadioButtons
import numpy as np
import pandas as pd
import sys

# Reservoirs with discharge data
reservoirs = [
    'castaic',
    'del_valle',
    'feather',
    'pyramid',
    'silverwood',
    'thermalito']

usgs_sites = dict(
    castaic=[11108092], 
    del_valle=[11176400], 
    feather=[11404500,11396200,11405200], 
    pyramid=[11109395, 11109375], 
    silverwood=[10260550, 10260700], 
    thermalito=[11406810, 11407000])

def load_usgs_csv(reservoir):
    if not reservoir in reservoirs:
        print('Reservoir not found')
        return
    
    num_sites = len(usgs_sites[reservoir])
    
    min_date = datetime.date(1900,1,1)
    max_date = datetime.date.today()
    
    dates = pd.date_range(min_date, max_date)
    num_dates = len(dates)
    discharge = pd.DataFrame(np.array([np.nan] * num_dates * num_sites).reshape(num_dates, num_sites), index=dates)
    
    site_index = 0
    for site_no in usgs_sites[reservoir]:
        filename='../data/'+reservoir+'/usgs_'+str(site_no)+'.csv'

        temp_discharge = pd.read_csv(filename, sep='\t', comment='#',
                                     na_values=['--','','m'], parse_dates=[2], 
                                     infer_datetime_format=True)[1:]
        
        if reservoir == 'thermalito' and site_index == 0:
            temp_discharge = temp_discharge.set_index('Date') # Reindex on the column 'Date'
        else:
            temp_discharge = temp_discharge.set_index('datetime') # Reindex on the column 'datetime'
        
        temp_discharge = temp_discharge.iloc[:,2].apply(float) # Grab only the discharge data
        
        if reservoir == 'thermalito' and site_index == 1:
            temp_discharge = -temp_discharge
        
        # Replace data in the discharge df with the dates from this site
        discharge.loc[temp_discharge.index[0]:temp_discharge.index[-1], site_index] = temp_discharge.values
        
        site_index += 1
    
    # Drop all rows with a NaN value and sum across columns
    discharge = discharge.dropna(how='any').sum(axis=1)
        
    return pd.to_datetime(discharge.index.get_values()), discharge

def find_index(dates, year):
    """Finds the index of `year` in `dates`.
    """
    index = 0
    for date in dates:
        if date.year == int(year):
            return index
        index += 1
    return -1

# Set defaults and load initial data
default_reservoir = 'feather'
dates, discharge = load_usgs_csv(default_reservoir)

min_date = dates[0]
max_date = dates[-1]

# Create the empty plot
fig = plt.figure(figsize=(10, 6), dpi=80)
ax = fig.gca()

# Shrink the plot by 25% towards upper-right corner,
# allowing room for the radio buttons to the left and
# the sliders below.
plt.subplots_adjust(left=0.25, bottom=0.25)
l, = plt.plot(dates, discharge, linewidth=2, color='black')

# Save for future use
plot_axes = fig.gca()

# Set the axis information
plt.xlabel('Date')
plt.ylabel('Discharge (cfs)')
ax.xaxis.set_major_formatter(DateFormatter('%Y-%m'))
ax.set_title(default_reservoir)

# Create the new axes for the Slider widgets
axcolor = 'lightgoldenrodyellow'
axyear_start = plt.axes([0.25, 0.125, 0.65, 0.03], facecolor=axcolor)
axyear_end = plt.axes([0.25, 0.085, 0.65, 0.03], facecolor=axcolor)

# Create the Slider widgets
syear_start = Slider(axyear_start, 'Start', min_date.year, max_date.year, valinit=min_date.year, valstep=1, valfmt='%1.f')
syear_end = Slider(axyear_end, 'End', min_date.year, max_date.year, valinit=max_date.year, valstep=1, valfmt='%1.f')

def update(val):
    """Called whenever either of the slider values is updated.
    """
    year_start = syear_start.val
    year_end = syear_end.val
    if not year_start < year_end:
        return
    s_index = find_index(dates, year_start)
    e_index = find_index(dates, year_end)
    l.set_xdata(dates[s_index:e_index])
    l.set_ydata(discharge[s_index:e_index])
    ax.relim()
    ax.autoscale_view()
    plt.draw()
syear_start.on_changed(update)
syear_end.on_changed(update)

resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', color=axcolor, hovercolor='0.975')

def reset(event):
    """Called whenever the reset button is clicked.
    """
    global syear_start, syear_end, site_no_radio, reservoir_radio, min_date, max_date, dates, discharge
    reservoir = default_reservoir
    # Load new data
    dates, discharge = load_usgs_csv(reservoir)
    min_date = dates[0]
    max_date = dates[-1]
    max_discharge = discharge.max()
    # Plot new data
    l.set_xdata(dates)
    l.set_ydata(discharge)
    ax.set_title(reservoir)
    ax.relim()
    ax.autoscale_view()
    # Reset sliders with new data
    axyear_start.clear()
    syear_start = Slider(axyear_start, 'Start', min_date.year, max_date.year, valinit=min_date.year, valstep=1, valfmt='%1.f')
    syear_start.on_changed(update)
    axyear_end.clear()
    syear_end = Slider(axyear_end, 'End', min_date.year, max_date.year, valinit=max_date.year, valstep=1, valfmt='%1.f')
    syear_end.on_changed(update)
    # Reset radio buttons
    rax.clear()
    reservoir_radio = RadioButtons(rax, reservoirs, active=reservoirs.index(default_reservoir))
    reservoir_radio.on_clicked(set_reservoir)
    fig.canvas.flush_events()
button.on_clicked(reset)

saveax = plt.axes([0.675, 0.025, 0.1, 0.04])
save_button = Button(saveax, 'Save PNG', color=axcolor, hovercolor='0.975')

def save_png(event):
    """Called whenever "Save PNG" is clicked."""
    # Get plot axes extent
    # See here for code: https://stackoverflow.com/questions/4325733/save-a-subplot-in-matplotlib
    extent = plot_axes.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    
    # Expanded the bbox to include the axes labels
    # 28% in the x-direction, 24% in the y-direction
    # Adjust as needed
    extent = extent.expanded(1.28, 1.24)
    
    # Save the figure
    fig.savefig("fig.png", bbox_inches=extent)
    print('Saved fig.png')
save_button.on_clicked(save_png)

rax = plt.axes([0.025, 0.35, 0.12, 0.3], facecolor=axcolor)
reservoir_radio = RadioButtons(rax, reservoirs, active=reservoirs.index(default_reservoir))

def set_reservoir(label):
    """Called when a new reservoir radio button is toggled.
    
    First, it loads the new data using the load_usgs_csv function
    from above. Then it sets the new data in the plot, clears all the axes 
    and recreates all the widgets from scratch.
    """
    global syear_start, syear_end, site_no_radio, min_date, max_date, dates, discharge
    reservoir = label
    # Load new data
    dates, discharge = load_usgs_csv(reservoir)
    min_date = dates[0]
    max_date = dates[-1]
    max_discharge = discharge.max()
    # Plot new data
    l.set_xdata(dates)
    l.set_ydata(discharge)
    ax.set_title(reservoir)
    ax.relim()
    ax.autoscale_view()
    # Reset sliders with new data
    axyear_start.clear()
    syear_start = Slider(axyear_start, 'Start', min_date.year, max_date.year, valinit=min_date.year, valstep=1, valfmt='%1.f')
    syear_start.on_changed(update)
    axyear_end.clear()
    syear_end = Slider(axyear_end, 'End', min_date.year, max_date.year, valinit=max_date.year, valstep=1, valfmt='%1.f')
    syear_end.on_changed(update)
    fig.canvas.flush_events()
reservoir_radio.on_clicked(set_reservoir)
    
plt.show()

FigureCanvasNbAgg()

Saved fig.png


## Summary statistics

The following shows the statistical summary of the data for each reservoir.

In [4]:
for reservoir in reservoirs:
    dates, discharge = load_usgs_csv(reservoir)
    print(reservoir)
    print(discharge.describe())
    print('\n')

castaic
count     6963.000000
mean     20984.061181
std       2993.103442
min       3264.000000
25%      19220.000000
50%      20760.000000
75%      22665.000000
max      31540.000000
dtype: float64


del_valle
count    20022.000000
mean        34.273061
std        172.688450
min          0.000000
25%          0.000000
50%          1.000000
75%          9.400000
max       4920.000000
dtype: float64


feather
count     9431.000000
mean       573.102553
std       2072.530616
min         37.600000
25%        105.500000
50%        146.100000
75%        174.700000
max      68091.000000
dtype: float64


pyramid
count     8764.000000
mean        44.163705
std        238.359077
min          0.660000
25%          4.400000
50%          8.800000
75%         21.000000
max      12173.000000
dtype: float64


silverwood
count    7304.000000
mean        7.471572
std        50.336807
min         0.000000
25%         0.000000
50%         0.650000
75%         3.100000
max      2730.000000
dtype: float64
